In [21]:
import numpy as np 
import pandas as pd 
from scipy.interpolate import interp1d

In [22]:
# importing and looking at the different countries
google_df = pd.read_csv('../raw_data/google_dataset(countries).csv')
jhuc_df = pd.read_csv('../raw_data/jhu_confirmed_dataset.csv')
jhud_df = pd.read_csv('../raw_data/jhu_deaths_dataset.csv')
jhur_df = pd.read_csv('../raw_data/jhu_recovered_dataset.csv')
oxford_df = pd.read_csv('../raw_data/oxford_dataset.csv')

In [25]:
len(jhud_df['Country/Region'].unique())

188

In [3]:
GOOGLE_LENGTH_OF_TIME = 92 # how much data we currently have for google 

In [4]:
google_df.head()

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [5]:
# drop all of the entries that are sub regions of countries
google_df = google_df[google_df['sub_region_1'].isna()]
google_df = google_df[google_df['sub_region_2'].isna()]

In [6]:
len(google_df['country_region_code'].unique())

132

In [7]:
# find the bad values 
google_df[google_df['retail_and_recreation_percent_change_from_baseline'].isnull()].to_csv('hand_analyze.csv')

# looks like the empty ones are all in Guinea-Bissau, Liechtenstein, and Reunion, lets just drop those...
google_df = google_df[google_df['country_region_code'] != 'GW']
google_df = google_df[google_df['country_region_code'] != 'LI']
google_df = google_df[google_df['country_region_code'] != 'RE']

In [8]:
#google_df[google_df['grocery_and_pharmacy_percent_change_from_baseline'].isnull()].to_csv('hand_analyze.csv')
# need to interpolate some vals 

In [9]:
#google_df[google_df['parks_percent_change_from_baseline'].isnull()].to_csv('hand_analyze.csv')
# none 

In [10]:
#google_df[google_df['transit_stations_percent_change_from_baseline'].isnull()].to_csv('hand_analyze.csv')
# none

In [11]:
#google_df[google_df['workplaces_percent_change_from_baseline'].isnull()].to_csv('hand_analyze.csv')
# none

In [12]:
#google_df[google_df['residential_percent_change_from_baseline'].isnull()].to_csv('hand_analyze.csv')
# empty values, but can interpoloate these 

In [13]:
def interpolate_data(transit_data):
    if np.count_nonzero(transit_data) <= 1: # absolutely no data or one data point 
        # print("no data to interpolate")
        a = np.empty(len(transit_data))
        a[:] = np.nan
        return a
    indice, = transit_data.nonzero()
    start, stop = indice[0], indice[-1]+1

    x = np.arange(len(transit_data))
    f = interp1d(x[indice], transit_data[indice])
    transit_data[start:stop] = f(x[start:stop])
    return transit_data

In [14]:
google_df.fillna(0, inplace = True)

In [15]:
# need to interpolate the grocery and residential data


for country_code in google_df['country_region_code'].unique(): 
    ts_data = np.array(google_df['residential_percent_change_from_baseline']\
                            [google_df['country_region_code'] == country_code])
    if len(ts_data) != GOOGLE_LENGTH_OF_TIME:
        google_df.drop(np.where(google_df['country_region_code'] == country_code)[0], inplace = True)
        google_df.reset_index(drop = True, inplace = True)
    else: 
        google_df['residential_percent_change_from_baseline'][google_df['country_region_code'] == country_code] \
        = interpolate_data(ts_data)
        
        

/Users/thomastran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [16]:
google_df['residential_percent_change_from_baseline'].isnull().sum()

0

In [17]:
# need to interpolate the grocery and residential data


for country_code in google_df['country_region_code'].unique(): 
    ts_data = np.array(google_df['grocery_and_pharmacy_percent_change_from_baseline']\
                            [google_df['country_region_code'] == country_code])
    if len(ts_data) != GOOGLE_LENGTH_OF_TIME:
        google_df.drop(np.where(google_df['country_region_code'] == country_code)[0], inplace = True)
        google_df.reset_index(drop = True, inplace = True)
    else: 
        google_df['grocery_and_pharmacy_percent_change_from_baseline'][google_df['country_region_code'] == country_code] \
        = interpolate_data(ts_data)
        

/Users/thomastran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [20]:
len(google_df['country_region_code'].unique())

129

In [18]:
google_df['grocery_and_pharmacy_percent_change_from_baseline'].isnull().sum()

0

In [19]:
google_df.to_csv('../processed_data/google_dataset.csv')